In [70]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import datetime
import math

# Bước 1: Reading dataframe

In [71]:
sheet_id = '1FLY9x5xmQA5C4b9DQeMaEH_21PNK-hW7'
sheet_name = '1_DEMAND'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_demand = pd.read_csv(url)
# đơn vị thành phẩm: tấn

In [72]:
sheet_name = '2_BOM'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_BOM = pd.read_csv(url, usecols=[0,1,2])
# đơn vị là kg (ngô, khoai, sắn, xương động vật, Bột đỗ tương, thóc), cái (bao bì), lít (men vi sinh)

In [73]:
sheet_name = '3_SUPPLY-SOH'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_SUPPLY_SOH = pd.read_csv(url, usecols=[0,1,3])
# đơn vị là kg (ngô, khoai, sắn, xương động vật, Bột đỗ tương, thóc), cái (bao bì), lít (men vi sinh)

# converting datetime
df_SUPPLY_SOH['HẠN SỬ DỤNG'] = pd.to_datetime(df_SUPPLY_SOH['HẠN SỬ DỤNG'], format='%d-%b-%Y')

In [74]:
sheet_name = '4_SUPPLY-ONGOING'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_SUPPLY_ONGOING = pd.read_csv(url, usecols=[0,2,3,5])
# đơn vị là kg (ngô, khoai, sắn, xương động vật, Bột đỗ tương, thóc), cái (bao bì), lít (men vi sinh)

# converting datetime
df_SUPPLY_ONGOING['NGÀY VỀ'] = pd.to_datetime(df_SUPPLY_ONGOING['NGÀY VỀ'], format='%d-%b-%Y')
df_SUPPLY_ONGOING['HẠN SỬ DỤNG'] = pd.to_datetime(df_SUPPLY_ONGOING['HẠN SỬ DỤNG'], format='%d-%b-%Y')

In [75]:
sheet_name = '5_ITEM MASTER - RAW'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
url = url.replace(" ", "%20")
df_ITEM_MASTER_RAW = pd.read_csv(url, usecols=[0,2,3,4,6])
df_ITEM_MASTER_RAW.rename(columns={'ITEM':'NGUYÊN LIỆU'}, inplace=True)
df_ITEM_MASTER_RAW.fillna(365, inplace=True)

In [76]:
sheet_name = '6_ITEM MASTER - FG'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
url = url.replace(" ", "%20")
df_ITEM_MASTER_FG = pd.read_csv(url, usecols=[0,2,3])

# Bước 2: Tìm số lượng nguyên liệu dự đoán theo nhu cầu dự đoán

## 2.1.  Đẩy lượng hàng thành phẩm còn tồn trong kho từ cuối tháng 12

In [77]:
# Lượng hàng thành phẩm còn tồn trong kho từ cuối tháng 12 sẽ được dùng để cung ứng cho tháng đầu tiên theo quy định số ngày tồn kho trung bình tối đa

# merge df_demand với df_SUPPLY_SOH để tìm số lượng hàng thành phẩm đang có trong kho
df_demand = df_demand.merge(df_SUPPLY_SOH[['ITEM', 'TỒN KHO CUỐI NGÀY 31/12/2022']], on='ITEM', how='left')

# số lượng hàng thành phẩm còn tồn trong kho sẽ được trừ trực tiếp vào demand của tháng 1
df_demand['JAN_2023'] = df_demand['JAN_2023'] - df_demand['TỒN KHO CUỐI NGÀY 31/12/2022']/1000

# xóa cột 'TỒN KHO CUỐI NGÀY 31/12/2022'
df_demand.drop(columns=['TỒN KHO CUỐI NGÀY 31/12/2022'], inplace=True)
df_demand

,ITEM,JAN_2023,FEB_2023,MAR_2023,APR_2023,MAY_2023,JUN_2023
0,THỨC_ĂN_CHO_HEO,11.0,22,30,25,24,27
1,THỨC_ĂN_CHO_CÁ,8.0,21,22,23,23,26


## 2.2. Unpivot df_demand

In [78]:
# unpivot df_demand
df_demand_melt = df_demand.melt('ITEM')
df_demand_melt = df_demand_melt.rename(columns = {'ITEM':'THÀNH PHẨM',
                                 'variable':'forecast_time',
                                 'value':'monthly_demand_qty'})

# converting datetime
df_demand_melt['forecast_time'] = pd.to_datetime(df_demand_melt['forecast_time'], format='%b_%Y')

# sắp xếp lại theo 'THÀNH PHẨM'
df_demand_melt = df_demand_melt.sort_values('forecast_time',ignore_index=True)
df_demand_melt

,THÀNH PHẨM,forecast_time,monthly_demand_qty
0,THỨC_ĂN_CHO_HEO,2023-01-01,11.0
1,THỨC_ĂN_CHO_CÁ,2023-01-01,8.0
2,THỨC_ĂN_CHO_HEO,2023-02-01,22.0
3,THỨC_ĂN_CHO_CÁ,2023-02-01,21.0
4,THỨC_ĂN_CHO_HEO,2023-03-01,30.0
5,THỨC_ĂN_CHO_CÁ,2023-03-01,22.0
6,THỨC_ĂN_CHO_HEO,2023-04-01,25.0
7,THỨC_ĂN_CHO_CÁ,2023-04-01,23.0
8,THỨC_ĂN_CHO_HEO,2023-05-01,24.0
9,THỨC_ĂN_CHO_CÁ,2023-05-01,23.0


## 2.3. Tìm sản lượng sản xuất theo từng ngày dựa theo df_demand (giả thiết: để đáp ứng nhu cầu của tháng đó thì mỗi ngày công ty sẽ sản xuất một lượng bằng nhu cầu/tổng số ngày trong tháng)

In [79]:
# tạo df_daily_production dựa theo df_demand_melt
df_daily_production = df_demand_melt
df_daily_production = df_daily_production.rename(columns={'monthly_demand_qty':'daily_production_qty'})

# tạo ra các ngày tương ứng trong từng tháng + thêm vào df_daily_production; tính toán daily_production_qty tương ứng
for i, data in df_demand_melt.iterrows():
    x = df_demand_melt.loc[i, 'forecast_time']
    df_daily_production.loc[i, 'daily_production_qty'] = df_demand_melt.loc[i, 'monthly_demand_qty'] / pd.Period(str(x)).days_in_month
    
    for j in range(pd.Period(str(x)).days_in_month - 1):
        df_daily_production.loc[len(df_daily_production)] = [df_demand_melt.loc[i,'THÀNH PHẨM'],
                                                   x + datetime.timedelta(days = j + 1),
                                                   df_demand_melt.loc[i, 'monthly_demand_qty'] / pd.Period(str(x)).days_in_month]
        
df_daily_production = df_daily_production.sort_values(by=['THÀNH PHẨM', 'forecast_time'], ignore_index=True)
df_daily_production

,THÀNH PHẨM,forecast_time,daily_production_qty
0,THỨC_ĂN_CHO_CÁ,2023-01-01,0.258065
1,THỨC_ĂN_CHO_CÁ,2023-01-02,0.258065
2,THỨC_ĂN_CHO_CÁ,2023-01-03,0.258065
3,THỨC_ĂN_CHO_CÁ,2023-01-04,0.258065
4,THỨC_ĂN_CHO_CÁ,2023-01-05,0.258065
...,...,...,...
357,THỨC_ĂN_CHO_HEO,2023-06-26,0.900000
358,THỨC_ĂN_CHO_HEO,2023-06-27,0.900000
359,THỨC_ĂN_CHO_HEO,2023-06-28,0.900000
360,THỨC_ĂN_CHO_HEO,2023-06-29,0.900000


## 2.4. Dựa vào leadtime sản xuất để xác định ngày bắt đầu sản xuất

In [80]:
# merge df_daily_production với dataframe chứa leadtime ngày sản xuất
df_daily_production = df_daily_production.merge(df_ITEM_MASTER_FG[['SKU', 'LEADTIME SẢN XUẤT \n(NGÀY)']],
                                                      left_on='THÀNH PHẨM',
                                                      right_on='SKU',
                                                      how='left')

# dùng vòng lặp để xác định số ngày bắt đầu sản xuất - job_start_date
for i, data in df_daily_production.iterrows():
    df_daily_production.loc[i, 'job_start_date'] = df_daily_production.loc[i, 'forecast_time'] - datetime.timedelta(days=int(df_daily_production.loc[i, 'LEADTIME SẢN XUẤT \n(NGÀY)']))

# Bỏ 3 cột 'forecast_time', 'SKU' và 'LEADTIME SẢN XUẤT \n(NGÀY)' khỏi dataframeT
df_daily_production.drop(columns=['forecast_time', 'SKU', 'LEADTIME SẢN XUẤT \n(NGÀY)'], inplace=True)
df_daily_production

,THÀNH PHẨM,daily_production_qty,job_start_date
0,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-01
1,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-02
2,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-03
3,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-04
4,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-05
...,...,...,...
357,THỨC_ĂN_CHO_HEO,0.900000,2023-06-26
358,THỨC_ĂN_CHO_HEO,0.900000,2023-06-27
359,THỨC_ĂN_CHO_HEO,0.900000,2023-06-28
360,THỨC_ĂN_CHO_HEO,0.900000,2023-06-29


## 2.5. Tìm số lượng nguyên liệu cần để sản xuất theo từng ngày theo từng thành phẩm

In [81]:
# merge df_daily_production và df_BOM
df_daily_production_merged = df_daily_production.merge(df_BOM, how='right', on='THÀNH PHẨM', sort=True)

# Xác định số lượng nguyên liệu cần để sản xuất theo từng ngày
df_daily_production_merged['daily_demand_raw_qty'] = df_daily_production_merged['daily_production_qty'] * df_daily_production_merged['SỐ LƯỢNG NGUYÊN LIỆU \n(TRÊN 1 TẤN THÀNH PHẨM)']
df_daily_production_merged

,THÀNH PHẨM,daily_production_qty,job_start_date,NGUYÊN LIỆU,SỐ LƯỢNG NGUYÊN LIỆU \n(TRÊN 1 TẤN THÀNH PHẨM),daily_demand_raw_qty
0,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-01,NGÔ,500,129.032258
1,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-02,NGÔ,500,129.032258
2,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-03,NGÔ,500,129.032258
3,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-04,NGÔ,500,129.032258
4,THỨC_ĂN_CHO_CÁ,0.258065,2023-01-05,NGÔ,500,129.032258
...,...,...,...,...,...,...
2167,THỨC_ĂN_CHO_HEO,0.900000,2023-06-26,BAO_BÌ_ĐÓNG_GÓI,40,36.000000
2168,THỨC_ĂN_CHO_HEO,0.900000,2023-06-27,BAO_BÌ_ĐÓNG_GÓI,40,36.000000
2169,THỨC_ĂN_CHO_HEO,0.900000,2023-06-28,BAO_BÌ_ĐÓNG_GÓI,40,36.000000
2170,THỨC_ĂN_CHO_HEO,0.900000,2023-06-29,BAO_BÌ_ĐÓNG_GÓI,40,36.000000


## 2.6. Tìm tổng số lượng nguyên liệu cần để sản xuất theo từng ngày

In [82]:
# cumsum 'daily_demand_raw_qty' theo từng ngày và theo 'NGUYÊN LIỆU'
df_daily_production_merged['cumsum_daily_demand_raw_qty'] = df_daily_production_merged.groupby(by=[df_daily_production_merged['job_start_date'],
                                                                                            df_daily_production_merged['NGUYÊN LIỆU']])['daily_demand_raw_qty'].cumsum()

# lấy giá trị cuối cùng khi groupby theo từng ngày và từng nguyên liệu để biết được tổng số lượng cần của 1 nguyên liệu trong 1 ngày
df_daily_production_merged = df_daily_production_merged.groupby(by=[df_daily_production_merged['job_start_date'], df_daily_production_merged['NGUYÊN LIỆU']],
                                                                group_keys=False,
                                                                sort=False).last()

# reset index của df_daily_production_merged
df_daily_production_merged.reset_index(inplace=True)

# loại bỏ những cột không còn sử dụng nữa trong df_daily_production_merged
df_daily_production_merged.drop(columns=['THÀNH PHẨM',
                                         'daily_production_qty',
                                         'daily_demand_raw_qty',
                                         'SỐ LƯỢNG NGUYÊN LIỆU \n(TRÊN 1 TẤN THÀNH PHẨM)'], inplace=True)
df_daily_production_merged

,job_start_date,NGUYÊN LIỆU,cumsum_daily_demand_raw_qty
0,2023-01-01,NGÔ,270.967742
1,2023-01-02,NGÔ,270.967742
2,2023-01-03,NGÔ,270.967742
3,2023-01-04,NGÔ,270.967742
4,2023-01-05,NGÔ,270.967742
...,...,...,...
1443,2023-06-26,SẮN,180.000000
1444,2023-06-27,SẮN,180.000000
1445,2023-06-28,SẮN,180.000000
1446,2023-06-29,SẮN,180.000000


# Bước 3: Tạo df_supply dùng cho kỳ đầu tiên từ df_SUPPLY_SOH và df_SUPPLY_ONGOING

## 3.1. Format df_SUPPLY_SOH

In [83]:
# Đổi tên cột
df_SUPPLY_SOH.rename(columns={'TỒN KHO CUỐI NGÀY 31/12/2022':'SỐ LƯỢNG',
                             'ITEM':'NGUYÊN LIỆU'}, inplace=True)
# Thêm 1 cột là 'NGÀY VỀ' và giá trị là '2000-1-1' để đáp ứng mục đích sort_values
df_SUPPLY_SOH.insert(1, column='NGÀY VỀ', value=pd.to_datetime('2000-1-1'))

df_SUPPLY_SOH

,NGUYÊN LIỆU,NGÀY VỀ,SỐ LƯỢNG,HẠN SỬ DỤNG
0,KHOAI,2000-01-01,10000,2023-02-12
1,SẮN,2000-01-01,3000,2023-02-27
2,XƯƠNG_ĐỘNG_VẬT,2000-01-01,2000,2023-01-26
3,BỘT_ĐỖ_TƯƠNG,2000-01-01,9000,2023-02-20
4,MEN_VI_SINH,2000-01-01,200,2023-01-25
5,NGÔ,2000-01-01,12000,2023-02-02
6,THÓC,2000-01-01,13000,2023-03-03
7,BAO_BÌ_ĐÓNG_GÓI,2000-01-01,1220,NaT
8,THỨC_ĂN_CHO_HEO,2000-01-01,7000,2023-01-27
9,THỨC_ĂN_CHO_CÁ,2000-01-01,8000,2023-01-30


## 3.2. Concat df_SUPPLY_SOH và df_SUPPLY_ONGOING

In [84]:
df_supply = pd.concat([df_SUPPLY_SOH, df_SUPPLY_ONGOING], axis=0)

# Sắp xếp dữ liệu theo 'ITEM' và theo 'NGÀY VỀ', không cần sắp xếp theo 'HẠN SỬ DỤNG' vì chắc chắn HSD của SOH luôn nhỏ hơn HSD của ONGOING SUPPLY
df_supply = df_supply.sort_values(by=['NGUYÊN LIỆU', 'NGÀY VỀ'])

# Đặt HSD của BAO_BÌ_ĐÓNG_GÓI là '2100-1-1' vì BAO_BÌ_ĐÓNG_GÓI không có HSD
df_supply.fillna(pd.to_datetime('2100-1-1'), inplace=True)
df_supply

,NGUYÊN LIỆU,NGÀY VỀ,SỐ LƯỢNG,HẠN SỬ DỤNG
7,BAO_BÌ_ĐÓNG_GÓI,2000-01-01,1220,2100-01-01
3,BỘT_ĐỖ_TƯƠNG,2000-01-01,9000,2023-02-20
0,KHOAI,2000-01-01,10000,2023-02-12
0,KHOAI,2023-01-03,1800,2023-04-13
4,MEN_VI_SINH,2000-01-01,200,2023-01-25
5,NGÔ,2000-01-01,12000,2023-02-02
1,NGÔ,2023-01-05,1000,2023-04-15
1,SẮN,2000-01-01,3000,2023-02-27
2,SẮN,2023-01-12,1000,2023-04-22
6,THÓC,2000-01-01,13000,2023-03-03


# Bước 4: Duyệt qua từng kỳ để tìm số lượng nguyên liệu cần đặt theo MOQ

## 4.1. Tìm ra số kỳ đặt hàng

In [85]:
# Tìm ra tổng số ngày sản xuất
total_days = df_daily_production_merged['job_start_date'].max() - df_daily_production_merged['job_start_date'].min()
total_days = total_days / datetime.timedelta(days=1)

# age_of_inventory = Số ngày tồn kho trung bình tối đa của các mặt hàng 
# (Biến này có thể thay đổi tùy theo quy định của Ban giám đốc)
age_of_inventory = 30

# Tìm ra số kỳ đặt hàng: nper
nper = math.ceil(total_days/age_of_inventory)

## 4.2. Dùng vòng lặp để xác định số lượng nguyên liệu cần đặt theo MOQ

In [86]:
def process_supply(df_in):
    # Nếu hết HẠN SỬ DỤNG thì SỐ LƯỢNG nguyên liệu supply sẽ bằng 0
    df_in.loc[df_in[df_in['job_start_date'] >= df_in['HẠN SỬ DỤNG']].index,'SỐ LƯỢNG'] = 0
    # Nếu hàng chưa về thì SỐ LƯỢNG nguyên liệu supply sẽ bằng 0
    df_in.loc[df_in[df_in['job_start_date'] < df_in['NGÀY VỀ']].index,'SỐ LƯỢNG'] = 0
    # Nếu hàng đã lưu kho quá số ngày tồn kho trung bình thì SỐ LƯỢNG nguyên liệu supply sẽ bằng 0
    df_in.loc[df_in[(df_in['job_start_date'] - df_in['NGÀY VỀ'] > pd.Timedelta(days=age_of_inventory))
                           & (df_in['NGÀY VỀ'] != pd.to_datetime('2000-1-1'))].index,'SỐ LƯỢNG'] = 0
    
    # Cumsum supply theo từng ngày và theo từng nguyên liệu
    df_in['cumsum_raw_supply'] = df_in.groupby(['job_start_date', 'NGUYÊN LIỆU'])['SỐ LƯỢNG'].cumsum()
    
    return df_in

In [87]:
def process_insufficient_qty(df_in):
    # xác định ngày đầu tiên bị thiếu hàng và lượng hàng bị thiếu
    df_in.loc[df_in['insufficient_qty'].lt(0).idxmax(),'insufficient_qty'] = -(df_in.loc[df_in['insufficient_qty'].lt(0).idxmax(),'insufficient_qty'])
    df_in.loc[df_in['insufficient_qty'].lt(0), 'insufficient_qty'] = df_in.loc[df_in['insufficient_qty'].lt(0), 'cumsum_daily_demand_raw_qty']
    # tính tổng lượng hàng bị thiếu
    df_in['order_qty'] = df_in['insufficient_qty'].sum()
    
    return df_in

In [88]:
def process_insufficient_first(df_in):
    for index, data in df_in.iterrows():
        # Xác định lượng đặt hàng theo MOQ và lượng hàng còn dư sau khi sử dụng
        if df_in.loc[index, 'order_qty'] >= df_in.loc[index, 'MOQ']:
            df_in.loc[index, 'MOQ_order_qty'] = df_in.loc[index, 'order_qty']
            df_in.loc[index, 'SỐ LƯỢNG'] = 0
        else:
            df_in.loc[index, 'MOQ_order_qty'] = df_in.loc[index, 'MOQ']
            df_in.loc[index, 'SỐ LƯỢNG'] = df_in.loc[index, 'MOQ_order_qty'] - df_in.loc[index, 'order_qty']

        # Xác định ngày đặt hàng
        df_in.loc[index, 'order_date'] = df_in.loc[index, 'job_start_date'] - pd.Timedelta(days=df_in.loc[index, 'LEADTIME ĐẶT HÀNG (NGÀY)'])

        # Xác định HẠN SỬ DỤNG của hàng order
        df_in.loc[index, 'HẠN SỬ DỤNG'] = df_in.loc[index, 'order_date'] + pd.Timedelta(days=df_in.loc[index, 'SHELF LIFE ĐƯỢC ƯỚC TÍNH KỂ TỪ NGÀY NHẬP KHO (NGÀY)'])
    
    return df_in

In [89]:
def process_df_insufficient(df_in, df_MOQ_order, df_supply):
    # Lấy ra giá trị cuối cùng theo từng nguyên liệu và từng ngày để xác định nguyên liệu bị thiếu supply
    df_insufficient_last = df_in.groupby(by=[df_in['job_start_date'], df_in['NGUYÊN LIỆU']],
                                             group_keys=False,
                                             sort=False).last()
    df_insufficient_last.reset_index(inplace=True)
    
    # Xác định những nguyên liệu bị thiếu
    df_insufficient_last = df_insufficient_last[df_insufficient_last['cumsum_raw_demand'] > df_insufficient_last['cumsum_raw_supply']]
    if len(df_insufficient_last) == 0:
        return df_MOQ_order, df_supply
    
    # Xác định lượng nguyên liệu bị thiếu theo từng ngày
    df_insufficient_last['insufficient_qty'] = df_insufficient_last['cumsum_raw_supply'] - df_insufficient_last['cumsum_raw_demand']
    df_insufficient_last.loc[df_insufficient_last[df_insufficient_last['cumsum_raw_supply']==0].index,'insufficient_qty'] = df_insufficient_last.loc[df_insufficient_last[df_insufficient_last['cumsum_raw_supply']==0].index,'cumsum_daily_demand_raw_qty']
    
    # Xác định tổng số lượng nguyên liệu bị thiếu cần phải đặt hàng
    df_insufficient_last = df_insufficient_last.groupby('NGUYÊN LIỆU').apply(process_insufficient_qty)
    
    # Tìm ngày đầu tiên bị thiếu hàng để đặt toàn bộ lượng hàng bị thiếu vào ngày đó
    df_insufficient_first = df_insufficient_last.groupby(by=[df_insufficient_last['NGUYÊN LIỆU'], df_insufficient_last['order_qty']],
                                                         group_keys=False,
                                                         sort=False).first()
    df_insufficient_first.reset_index(inplace=True)
    df_insufficient_first = df_insufficient_first[['NGUYÊN LIỆU', 'order_qty', 'job_start_date']]
    
    # Merge với df_ITEM_MASTER_RAW để tìm ra lượng đặt hàng theo MOQ
    df_insufficient_first = df_insufficient_first.merge(df_ITEM_MASTER_RAW, on='NGUYÊN LIỆU', how='left')
    df_insufficient_first = process_insufficient_first(df_insufficient_first)
    
    # Update df_MOQ_order
    df_MOQ_order = pd.concat([df_MOQ_order, df_insufficient_first[['NGUYÊN LIỆU', 'NHÀ CUNG CẤP', 'MOQ_order_qty', 'order_date']]], axis=0)
    
    # Update df_supply
    df_insufficient_first = df_insufficient_first[df_insufficient_first['SỐ LƯỢNG']!=0].rename(columns={'job_start_date':'NGÀY VỀ'})
    df_supply = pd.concat([df_supply, df_insufficient_first[['NGUYÊN LIỆU', 'NGÀY VỀ', 'SỐ LƯỢNG', 'HẠN SỬ DỤNG']]], axis=0)
    
    return df_MOQ_order, df_supply

In [90]:
# Tạo df_MOQ_order
df_MOQ_order = pd.DataFrame(columns=['NGUYÊN LIỆU', 'NHÀ CUNG CẤP', 'MOQ_order_qty', 'order_date'])

# lấy first_date_period là ngày đầu tiên bắt đầu sản xuất
first_date_period = df_daily_production_merged['job_start_date'].min()

for i in range(nper):
    # Xác định những ngày có trong từng kỳ
    period_days = [(first_date_period + datetime.timedelta(days=day)) for day in range(age_of_inventory+1)]
    # Xác định df_demand của từng kỳ
    df_demand = df_daily_production_merged[df_daily_production_merged['job_start_date'].isin(period_days)]
    
    # cumsum lượng nguyên liệu cần của 1 nguyên liệu theo từng kỳ
    df_demand['cumsum_raw_demand'] = df_demand.groupby(by=[df_demand['NGUYÊN LIỆU']])['cumsum_daily_demand_raw_qty'].cumsum()
    
    # Merge df_demand và df_supply
    df_merged = df_demand.merge(df_supply, how='left', on='NGUYÊN LIỆU')
    # Điền những giá trị NaN hoặc NaT do không có supply, đảm bảo logic supply không còn hàng
    df_merged['NGÀY VỀ'].fillna(pd.to_datetime('2000-1-1'), inplace=True)
    df_merged['SỐ LƯỢNG'].fillna(0,inplace=True)
    df_merged['HẠN SỬ DỤNG'].fillna(pd.to_datetime('2000-1-1'),inplace=True)
    
    # Xử lý supply sau khi merge
    df_merged = process_supply(df_merged)
    # Xử lý những nguyên liệu không đủ supply
    df_MOQ_order, df_supply = process_df_insufficient(df_merged, df_MOQ_order, df_supply)
    
    # Update df_supply
    df_supply = df_supply[df_supply['NGÀY VỀ'].isin(period_days)]
    df_supply = df_supply.sort_values(by=['NGUYÊN LIỆU', 'NGÀY VỀ'])
    
    # Update ngày đầu tiên của kỳ tiếp theo
    first_date_period = period_days[-1] + datetime.timedelta(days=1)

# Bước 5: Xuất ra file kết quả

In [91]:
# Pivot df_MOQ_order
df_MOQ_order_pivot = pd.pivot_table(df_MOQ_order,index=['NGUYÊN LIỆU', 'NHÀ CUNG CẤP'], columns=['order_date'], values=['MOQ_order_qty'])
df_MOQ_order_pivot.fillna('', inplace=True)
df_MOQ_order_pivot

MOQ_order_qty                          \
order_date                             2023-01-14 2023-01-23   2023-01-25   
NGUYÊN LIỆU     NHÀ CUNG CẤP                                                
BAO_BÌ_ĐÓNG_GÓI CÔNG TY TNHH WOPACK                           1921.290323   
BỘT_ĐỖ_TƯƠNG    CÔNG TY TNHH NARGI                            6938.709677   
KHOAI           CÔNG TY TNHH NARGI                                          
MEN_VI_SINH     CÔNG TY VNIBIO                         100.0                
NGÔ             CÔNG TY TNHH NARGI                                          
SẮN             CÔNG TY TNHH NARGI                                          
THÓC            CÔNG TY TNHH NARGI                            2312.903226   
XƯƠNG_ĐỘNG_VẬT  CÔNG TY CP YFOOD           1000.0             7293.548387   

                                                                             \
order_date                             2023-01-26   2023-01-27   2023-01-31   
NGUYÊN LIỆU     NHÀ CUNG CẤP                                                  
BAO_BÌ_ĐÓNG_GÓI CÔNG TY TNHH WOPACK                                           
BỘT_ĐỖ_TƯƠNG    CÔNG TY TNHH NARGI                                            
KHOAI           CÔNG TY TNHH NARGI                 4666.359447                
MEN_VI_SINH     CÔNG TY VNIBIO                                                
NGÔ             CÔNG TY TNHH NARGI   20525.806452                             
SẮN             CÔNG TY TNHH NARGI                              3980.645161   
THÓC            CÔNG TY TNHH NARGI                                            
XƯƠNG_ĐỘNG_VẬT  CÔNG TY CP YFOOD                                              

                                                                           \
order_date                           2023-02-02    2023-02-25  2023-03-02   
NGUYÊN LIỆU     NHÀ CUNG CẤP                                                
BAO_BÌ_ĐÓNG_GÓI CÔNG TY TNHH WOPACK               2070.709677               
BỘT_ĐỖ_TƯƠNG    CÔNG TY TNHH NARGI                6651.290323               
KHOAI           CÔNG TY TNHH NARGI                5919.354839               
MEN_VI_SINH     CÔNG TY VNIBIO       423.225806                517.677419   
NGÔ             CÔNG TY TNHH NARGI               22924.193548               
SẮN             CÔNG TY TNHH NARGI                5919.354839               
THÓC            CÔNG TY TNHH NARGI                2217.096774               
XƯƠNG_ĐỘNG_VẬT  CÔNG TY CP YFOOD                  8136.451613               

                                                                             \
order_date                             2023-03-28  2023-04-02    2023-04-28   
NGUYÊN LIỆU     NHÀ CUNG CẤP                                                  
BAO_BÌ_ĐÓNG_GÓI CÔNG TY TNHH WOPACK   1970.580645               1920.086022   
BỘT_ĐỖ_TƯƠNG    CÔNG TY TNHH NARGI    7100.322581               7049.677419   
KHOAI           CÔNG TY TNHH NARGI    5119.354839               4900.645161   
MEN_VI_SINH     CÔNG TY VNIBIO                     492.645161                 
NGÔ             CÔNG TY TNHH NARGI   22072.580645              21550.752688   
SẮN             CÔNG TY TNHH NARGI    5119.354839               4900.645161   
THÓC            CÔNG TY TNHH NARGI    2366.774194               2349.892473   
XƯƠNG_ĐỘNG_VẬT  CÔNG TY CP YFOOD      7486.129032               7250.537634   

                                                                           
order_date                           2023-05-03    2023-05-29  2023-06-03  
NGUYÊN LIỆU     NHÀ CUNG CẤP                                               
BAO_BÌ_ĐÓNG_GÓI CÔNG TY TNHH WOPACK               1837.333333              
BỘT_ĐỖ_TƯƠNG    CÔNG TY TNHH NARGI                     6760.0              
KHOAI           CÔNG TY TNHH NARGI                     4680.0              
MEN_VI_SINH     CÔNG TY VNIBIO       480.021505                459.333333  
NGÔ             CÔNG TY TNHH NARGI               20626.666667       

In [92]:
df_MOQ_order_pivot.to_excel('output.xlsx')